In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


uncomment bellow if you dont use google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install -U pip
#!pip3 install -U setuptools wheel
!pip3 install autogluon

In [ ]:
data=pd.read_csv('drive/MyDrive/immo_data_clean2.csv',lineterminator='\n')
handy_data=data.copy()

Deleting outliers

In [ ]:
def get_rid_outl(df, list_num_var, percentile):
    """
    INPUT: 
    df: Dataframe
    list_num_var: List of the variables (columns) with outliers that need to be deleted
    percentile: float between 0 and 100: Percentage of the data that will be kept 
    
    OUTPUT:
    DataFrame with the defined amount of data. This means a DataFrame without outliers
    """
    
    perc_dict = {}
    for col in list_num_var: # first, calculate all the percentiles before removing any
        # row, otherwise you will remove more rows than necessary
        value_perc = np.nanpercentile(df[col], percentile)
        value_perc_dict = {col: value_perc}
        perc_dict.update(value_perc_dict)
        
    for var in list_num_var:    
        df = df.loc[(df[var] <= perc_dict[var]) ^ (df[var].isnull())] # include nan values
    
    return df

Deleting unimportant columns

In [ ]:
handy_data.drop(columns=['regionLevel3'], inplace=True)

Considering just important part regionLevel2 as our previous model(xgboost) feature importance told us

In [ ]:
selec=['München','Stuttgart','Berlin','Frankfurt_am_Main','Düsseldorf','Köln','München_Kreis','Dresden','Mainz','Starnberg_Kreis']
all_region2=list(handy_data.regionLevel2.unique())
a={a:"noMatter" for a in all_region2 if a not in selec }
handy_data["regionLevel2"].replace(a, inplace=True)

In [ ]:
handy_data.loc[handy_data.yearConstructed == "NO_INFORMATION", "yearConstructed"] = '0'
handy_data['yearConstructed']=handy_data['yearConstructed'].astype(int)
handy_data.loc[handy_data.yearConstructed == 0, "yearConstructed"] = None
handy_data['yearConstructed'] = handy_data['yearConstructed'].fillna(handy_data['yearConstructed'].median())

In [ ]:
handy_data.drop(columns=['totalRent',
                 'serviceCharge',
                 'heatingCosts'], inplace=True)

# rename the response variable to rent
handy_data.rename(columns={'rent_incl_hc': 'rent'}, inplace=True)

# remove all rows where living space is 0
handy_data = handy_data[handy_data['livingSpace'] != 0]

# create a new variable for the rent/livingSpace (rent/m2)
handy_data['rent_m2'] = handy_data['rent'] / handy_data['livingSpace']

# get rid of outliers
handy_data = get_rid_outl(handy_data, ['rent_m2'], 99.8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
handy_data.typeOfFlat.replace(np.nan, 'NO_INFORMATION', inplace=True)
handy_data.interiorQuality.replace(np.nan, 'NO_INFORMATION', inplace=True)

In [ ]:
handy_data

In [ ]:
# first transform all categorical values into dummies
list_cat_vars = handy_data.select_dtypes(include = ['object']).columns
list_cat_vars_without_text=[feat for feat in list_cat_vars if feat not in ['description','facilities'] ]
for var in  list_cat_vars_without_text:
    # for each cat add dummy var, drop original column
    handy_data = pd.concat([handy_data.drop(var, axis=1), pd.get_dummies(handy_data[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)

handy_data.describe()

,Unnamed: 0,yearConstructed,noParkSpaces,livingSpace,postcode,noRooms,rent,rent_m2,regionLevel1_Bayern,regionLevel1_Berlin,...,lastRefurbish_2012,lastRefurbish_2013,lastRefurbish_2014,lastRefurbish_2015,lastRefurbish_2016,lastRefurbish_2017,lastRefurbish_2018,lastRefurbish_2019,lastRefurbish_2020,lastRefurbish_NO_INFORMATION
count,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,...,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000,217017.000000
mean,133744.051226,1966.855550,0.352903,70.732048,36454.058327,2.567579,774.526138,11.055926,0.079542,0.038845,...,0.009078,0.009603,0.013769,0.018132,0.022708,0.028454,0.051037,0.078072,0.014635,0.682933
std,77618.387396,42.499281,0.545758,26.514521,27942.024307,0.899271,424.700101,4.586531,0.270584,0.193226,...,0.094843,0.097523,0.116529,0.133430,0.148971,0.166266,0.220075,0.268286,0.120086,0.465335
min,0.000000,1000.000000,0.000000,1.000000,852.000000,1.000000,1.000000,0.009455,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,66499.000000,1957.000000,0.000000,53.630000,9119.000000,2.000000,473.820000,8.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,133359.000000,1972.000000,0.000000,66.210000,37574.000000,2.500000,645.000000,9.797297,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,200885.000000,1991.000000,1.000000,84.810000,55444.000000,3.000000,956.000000,12.691209,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,268848.000000,2020.000000,4.000000,180.000000,99998.000000,5.000000,2980.000000,45.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
handy_data_new=handy_data[[feat for feat in handy_data.columns.to_list() if feat not in ['Unnamed: 0']] ]
y = handy_data_new['rent']
X = handy_data_new[[i for i in handy_data_new.columns if i not in ['rent', 'rent_m2']]]
train, test = train_test_split(pd.concat([X,y],axis=1), test_size=0.2,random_state=42)

In [ ]:
train['facilities']

192241    Die Mietwohnung besteht aus 2 Zimmern und ist ...
131248    Die Wanne im Bad ist mit Duschabtrennung ausge...
64389                                                   NaN
155488    Fliesen- und Vinylboden, bodenebene Dusche, se...
87135     Die Wohnung verfügt über einen Balkon, von dem...
                                ...                        
120154    -komplett eingerichtete Küche\n-Waschmaschine/...
103942                                                  NaN
132230    Wohn, Flurbereich und Schlafzimmer sind mit La...
147190                                                  NaN
122236                                                  NaN
Name: facilities, Length: 173613, dtype: object

## Model : AutoGluon with german BERT

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
hyperparameters={'AG_IMAGE_NN': {},
 'AG_TEXT_NN': {'model.hf_text.checkpoint_name': 'bert-base-german-cased','optimization.max_epochs': 3},
 'CAT': {},
 'GBM': [{},
  {'ag_args': {'name_suffix': 'XT'}, 'extra_trees': True},
  'GBMLarge'],
 'NN_TORCH': {},
 'VW': {},
 'XGB': {}}

In [ ]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)
save_path = 'final_outs_agModels-predictClass_with_text'
label = 'rent'
predictor = TabularPredictor(label=label,eval_metric='r2', path=save_path).fit(train_data,hyperparameters=hyperparameters,ag_args_fit={'num_gpus': 1})
predictions = predictor.predict(test_data)
perf = predictor.evaluate_predictions(y_true=test_data['rent'], y_pred=predictions, auxiliary_metrics=True)
perf

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "final_outs_agModels-predictClass_with_text/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    173613
Train Data Columns: 179
Label Column: rent
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2980.0, 1.0, 775.3316, 425.15361)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Ava

[1000]	valid_set's l2: 18981.7	valid_set's r2: 0.901956
[2000]	valid_set's l2: 17979.2	valid_set's r2: 0.907113
[3000]	valid_set's l2: 17508.2	valid_set's r2: 0.909566
[4000]	valid_set's l2: 17238.8	valid_set's r2: 0.910958
[5000]	valid_set's l2: 17044.3	valid_set's r2: 0.911951
[6000]	valid_set's l2: 16835.1	valid_set's r2: 0.913035
[7000]	valid_set's l2: 16755.2	valid_set's r2: 0.913456
[8000]	valid_set's l2: 16700.7	valid_set's r2: 0.913718
[9000]	valid_set's l2: 16592.7	valid_set's r2: 0.914291
[10000]	valid_set's l2: 16548.2	valid_set's r2: 0.914523


	0.9145	 = Validation score   (r2)
	204.73s	 = Training   runtime
	0.52s	 = Validation runtime
Fitting model: LightGBMXT ...
	Training LightGBMXT with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	valid_set's l2: 19753.9	valid_set's r2: 0.897955
[2000]	valid_set's l2: 18906.5	valid_set's r2: 0.902341
[3000]	valid_set's l2: 18464.8	valid_set's r2: 0.904625
[4000]	valid_set's l2: 18241.8	valid_set's r2: 0.905782
[5000]	valid_set's l2: 18034.2	valid_set's r2: 0.906852
[6000]	valid_set's l2: 17857.3	valid_set's r2: 0.907757
[7000]	valid_set's l2: 17700.4	valid_set's r2: 0.90857
[8000]	valid_set's l2: 17629.6	valid_set's r2: 0.908938
[9000]	valid_set's l2: 17557.5	valid_set's r2: 0.909303
[10000]	valid_set's l2: 17500.7	valid_set's r2: 0.909597


	0.9097	 = Validation score   (r2)
	201.74s	 = Training   runtime
	0.55s	 = Validation runtime
Fitting model: CatBoost ...
	Training CatBoost with GPU, note that this may negatively impact model quality compared to CPU training.
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
	0.8981	 = Validation score   (r2)
	62.05s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: XGBoost ...
	0.9161	 = Validation score   (r2)
	248.89s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8577	 = Validation score   (r2)
	164.82s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: VowpalWabbit ...
		`import vowpalwabbit` failed.
A quick tip is to install via `pip install vowpalwabbit==8.10.1
Fitting model: ImagePredictor ...
	No valid features to train ImagePredictor... Skipping this model.
Fitting model: TextPredictor ...
Global seed set to 0
Using 16bit native Autom

...

Predicting: 0it [00:00, ?it/s]

Fitting model: WeightedEnsemble_L2 ...
	0.92	 = Validation score   (r2)
	0.25s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 31141.39s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("final_outs_agModels-predictClass_with_text/")
Load pretrained checkpoint: final_outs_agModels-predictClass_with_text/models/TextPredictor/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

Evaluation: r2 on test data: 0.92394554377936
Evaluations on test data:
{
    "r2": 0.92394554377936,
    "root_mean_squared_error": -116.6178388808218,
    "mean_squared_error": -13599.720345234406,
    "mean_absolute_error": -75.077012793151,
    "pearsonr": 0.9612937039801384,
    "median_absolute_error": -45.924224853515625
}


{'mean_absolute_error': -75.077012793151,
 'mean_squared_error': -13599.720345234406,
 'median_absolute_error': -45.924224853515625,
 'pearsonr': 0.9612937039801384,
 'r2': 0.92394554377936,
 'root_mean_squared_error': -116.6178388808218}

In [ ]:
!zip -r final_outs_agModels-predictClass_with_text.zip final_outs_agModels-predictClass_with_text

  adding: final_outs_agModels-predictClass_with_text/ (stored 0%)
  adding: final_outs_agModels-predictClass_with_text/predictor.pkl (deflated 30%)
  adding: final_outs_agModels-predictClass_with_text/utils/ (stored 0%)
  adding: final_outs_agModels-predictClass_with_text/utils/data/ (stored 0%)
  adding: final_outs_agModels-predictClass_with_text/utils/data/X_val.pkl (deflated 86%)
  adding: final_outs_agModels-predictClass_with_text/utils/data/y.pkl (deflated 64%)
  adding: final_outs_agModels-predictClass_with_text/utils/data/y_val.pkl (deflated 62%)
  adding: final_outs_agModels-predictClass_with_text/utils/data/X.pkl (deflated 87%)
  adding: final_outs_agModels-predictClass_with_text/learner.pkl (deflated 74%)
  adding: final_outs_agModels-predictClass_with_text/__version__ (stored 0%)
  adding: final_outs_agModels-predictClass_with_text/models/ (stored 0%)
  adding: final_outs_agModels-predictClass_with_text/models/trainer.pkl (deflated 80%)
  adding: final_outs_agModels-predictC

In [ ]:
!cp -r final_outs_agModels-predictClass_with_text drive/MyDrive

In [ ]:
predictor.leaderboard(test_data)

Load pretrained checkpoint: final_outs_agModels-predictClass_with_text/models/TextPredictor/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

                 model  score_test  score_val  pred_time_test  pred_time_val      fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2    0.923946   0.920025      882.696298      52.246099  30387.660108                 0.019004                0.003047           0.246047            2       True          8
1        LightGBMLarge    0.921008   0.915679       17.711222       0.916459    319.535500                17.711222                0.916459         319.535500            1       True          7
2             LightGBM    0.918580   0.914542       11.795542       0.523353    204.731900                11.795542                0.523353         204.731900            1       True          1
3              XGBoost    0.917659   0.916087        7.406900       0.404744    248.885429                 7.406900                0.404744         248.885429            1       True          4
4           LightGBMXT    0.91

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.923946,0.920025,882.696298,52.246099,30387.660108,0.019004,0.003047,0.246047,2,True,8
1,LightGBMLarge,0.921008,0.915679,17.711222,0.916459,319.535500,17.711222,0.916459,319.535500,1,True,7
2,LightGBM,0.918580,0.914542,11.795542,0.523353,204.731900,11.795542,0.523353,204.731900,1,True,1
3,XGBoost,0.917659,0.916087,7.406900,0.404744,248.885429,7.406900,0.404744,248.885429,1,True,4
4,LightGBMXT,0.914607,0.909650,12.182194,0.554874,201.738413,12.182194,0.554874,201.738413,1,True,2
5,CatBoost,0.902254,0.898082,0.832838,0.092703,62.046536,0.832838,0.092703,62.046536,1,True,3
6,NeuralNetTorch,0.858622,0.857672,0.525044,0.058088,164.816053,0.525044,0.058088,164.816053,1,True,5
7,TextPredictor,0.813160,0.814625,845.763629,50.398495,29614.261231,845.763629,50.398495,29614.261231,1,True,6


## For inference just load the weights(final_outs_agModels-predictClass_with_text drive) and uncomment bellow

In [ ]:
# predictor.load('final_outs_agModels-predictClass_with_text drive').leaderboard(test_data)